<a href="https://colab.research.google.com/github/TiffanyClark225/DES/blob/master/EE576_Bitcoin_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this assignment is to help you to learn Bitcoin Script in writing basic transactions and simple smart contracts. This assignment is based on a similar assignment from a graduate-level course on bitcoin and cryptocurrencies at Stanford University.


You should be familiar with Bitcoin script. Its basic structure was covered in class and chapter 3 of the book “[Bitcoin and Cryptocurrency Technologies](https://d28rh4a8wq0iu5.cloudfront.net/bitcointech/readings/princeton_bitcoin_book.pdf)”. Examples of some opcodes that you will likely be making use of include OP DUP, OP CHECKSIG, OP EQUALVERIFY, and OP CHECKMULTISIG, but you will end up using additional ones as well. Full documentation can be found [here](https://en.bitcoin.it/wiki/Script).

## Setup the environments

In [ ]:
#@title Download the libraries

!pip install python-bitcoinlib
!pip install forex-python

In [ ]:
#@title Setup our environment to use Bitcoin Testnet { form-width: "30%" }
import requests
from os import urandom
from bitcoin import SelectParams
from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.base58 import decode
from bitcoin.wallet import CBitcoinAddress, CBitcoinSecret, P2PKHBitcoinAddress
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

SelectParams('testnet')
print("Testnet selected.")

### To do (you can put your answers right here):


1.   Explain how the private key and addresses are used in Bitcoin. How do you know that this is an address for the Testnet blockchain but not the production Bitcoin block chain?
2.   Modify the code below so that you can reuse your private key.
3.   Go to a <a href="https://coinfaucet.eu/en/btc-testnet/">Testnet Faucet</a>, paste in your address to get some "free" money! We are going to spend this money in a while so we are going to call this unspent transaction output (UTXO). Find out the transaction ID, amount, and ```utxo_txid_index``` in the code below. What is the transaction fee, if any?
4.   If the bitcoin comes from the real Bitcoin network, how much money (USD) would you have received?


-- Put your answer here --

In [ ]:
## Question 2: modify the code so that you can reuse your private key (and hence your public key)
my_private_key = CBitcoinSecret.from_secret_bytes(urandom(32))
my_public_key  = my_private_key.pub
my_address     = P2PKHBitcoinAddress.from_pubkey(my_public_key)
print("My bitcoin address is %s" % my_address)

In [ ]:
## Questions 3 and 4
## To Do: Put in your transaction ID and amounts in the next two lines
utxo_amount     = 0.01463481
utxo_txid       = 'bf173dd728237d8a1a395048fb54a00b1a92e1c661ec9702d2554b5bc01fef09'

# Figure out how to use BtcConverter to determine the value of bitcoin
from forex_python.bitcoin import BtcConverter
b = BtcConverter()
print("If it was real bitcoin, you would be USD$%s richer!" % "0")

## To Do: Click on the transaction id to go to the block explorer. Scroll down to the details
## of the transaction and you should see two outputs. One output goes to you and the unspent
## output goes back to the faucet. If your output comes first, put in 0 below and 1 otherwise.
utxo_txid_index = 1

## Task 1. Direct Transfer Transaction

The first task is for you to transfer your bitcoins to another person. Every bitcoin transaction has an input and an output. The input is the reference transaction that indicates the source of the fund. In this case, this is the transaction ```utxo_txid``` where Testnet has created the UTXO for you.

There are two parts of the output. First, you need to unlock the UTXO from Testnet using an unlocking script or scriptSig. In this case, you need to match the public key hash and also sign with your private key. This kind of thing is done a lot in Bitcoin and the script used in this case is called pay-to-public-key-hash script.

Second, you need to set up the condition so that the intended recipient of your fund can unlock it. This part is called the scriptPubKey. To facilitate the redemption of the fund, you can also put the pay-to-public-key-hash script here as well so the recipient can easily redeem this by signing it off with his/her private key.

In [ ]:
#@title Define the transaction amount { form-width: "30%" }

output_amount = utxo_amount - 0.001
print("Give away %f BTC but leave %f BTC as transaction fee." % (output_amount, utxo_amount-output_amount))

### To do:

1.   Send the bitcoin to your partner
2.   Explain in your own words what pay-to-public-key-hash works
3.   If all goes well, write down the transaction ID.
4.   Copy and paste it to the block explorer and check if everything goes through.



In [ ]:
#@title Definition of ```scriptPubKey``` { form-width: "30%" }

# To do: define recipient_address as your partner's address. Hint: use function CBitcoinAddress
# recipient_address = ...

# pay-to-public-key-hash - this was explained in class
scriptPubKey   = [OP_DUP, OP_HASH160, recipient_address, OP_EQUALVERIFY, OP_CHECKSIG]

txout          = CMutableTxOut(output_amount*COIN, CScript(scriptPubKey))
print("Output looks like this : %s" % txout)

In [ ]:
#@title Link to the reference transaction where the source fund is { form-width: "30%" }
txin = CMutableTxIn(COutPoint(lx(utxo_txid), utxo_txid_index))
print("It's not complete as the script part is empty: %s" % txin)

In [ ]:
#@title Definition of "scriptSig" to unlock the reference transaction { form-width: "30%" }

# another pay-to-public-key-hash but this time it uses your own address
# Note that this script should match exactly the script PubKey from the testnet - this
# script is NOT part of what you post but you need to sign it for redemption
utxo_scriptPubKey = [OP_DUP, OP_HASH160, my_address, OP_EQUALVERIFY, OP_CHECKSIG]

# sign the whole transaction
tx_tmp         = CMutableTransaction([txin], [txout])
sighash        = SignatureHash(CScript(utxo_scriptPubKey), tx_tmp, 0, SIGHASH_ALL)
signature      = my_private_key.sign(sighash) + bytes([SIGHASH_ALL])
tx             = CMutableTransaction([txin], [txout])
txin.scriptSig = CScript([signature, my_public_key])
try:
  # VerifyScript checks if txin.scriptSign can redeem utxo_scriptPubKey
  VerifyScript(txin.scriptSig, CScript(utxo_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
  print("Script looks good, ready to go!")
except Exception:
  traceback.print_exc()


In [ ]:
#@title Post it to Testnet and see what happen
raw_transaction = b2x(tx.serialize())
headers         = {'content-type': 'application/x-www-form-urlencoded'}
response        = requests.post('https://api.blockcypher.com/v1/btc/test3/txs/push',
                                headers=headers,
                                data='{"tx": "%s"}' % raw_transaction)
print(response.status_code, response.reason)
print(response.text)

## Task 2. Smart Contract 1

Generate a transaction so that your partner will give the bitcoin to whoever can provide the solution $(x,y)$ to the following system of two linear equations:
   
   $x + y =$ first 2 digits of the larger student id in your team

and

   $x - y =$ first 2 digits of the smaller student id in your team

To ensure that an integer solution exists, please change the last digit of the two numbers on the right hand side so the numbers are both even or both odd.

Demonstrate your solution by developing the redemption script and get all the bitcoins back yourself. The redemption script should be as small as possible. That is, a valid script-sig should consist of simply pushing two integers $x$ and $y$ to the stack.

Hint: Make sure you use ```OP_ADD``` and ```OP_SUB``` in your script.


In [ ]:
# Input Script - get the bitcoin from your partner and give it to whoever can solve the problem
# Your code is below

# Step 0. Let's assume
xPlusy  = 890
xMinusy = 630

# Step 1. Define utxo_amount, utxo_txid, utxo_txid_index, and output_amount as in the example

# Step 2. sm1_scriptPubKey contains the actual contract based on Bitcoin Script
sm1_scriptPubKey = []

# Step 3. Formulate the transaction (no need to change)
txout             = CMutableTxOut(output_amount*COIN, CScript(sm1_scriptPubKey))
txin              = CMutableTxIn(COutPoint(lx(utxo_txid), utxo_txid_index))
tx                = CMutableTransaction([txin], [txout])

# Step 4. Another pay-to-public-key-hash but this time, replace @@@@@ with your partner address
utxo_scriptPubKey = [OP_DUP, OP_HASH160, @@@@@, OP_EQUALVERIFY, OP_CHECKSIG]

# Step 5. Follow the previous example but make sure to use your partner's private key to sign for it.

# Step 6. Verify before posting

# Step 7. Posting



In [ ]:
# Redemption Script - solve the problem by providing x and y, and then transfer the bitcoin back to yourself.
# Your code is below

# Step 0. What are x and y? Make sure that the order matches your sm1_scriptPubKey

# Step 1. Define utxo_amount, utxo_txid, utxo_txid_index, and output_amount as in the example

# Step 2. Script PubKey is similar to the previous example as you are getting back the bitcoins yourself

# Step 3. utxo_scriptPubKey contains the script to satisfy sm1_scriptPubKey
utxo_scriptPubKey = []

# Step 4. No need to sign for this transaction as anybody can solve and get the bitcoin (No need to change)
tx                = CMutableTransaction([txin], [txout])
txin.scriptSig    = CScript(utxo_scriptPubKey)

# Step 5. VerifyScript needs sm1_scriptPubKey to check everything is okay before posting (No need to change)
try:
  VerifyScript(txin.scriptSig, CScript(sm1_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
  print("Script looks good, ready to go!")
except Exception:
  traceback.print_exc()

# Step 6. Posting

## Task 3. Smart Contract 2

Generate a multi-sig transaction involving four parties such that the transaction can be redeemed by the first party (your partner) combined with any one of the 3 people (customers) created by you but not by only the customers or only your partner.

Demonstrate your work by writing a redemption script so that your partner can get back your bitcoins and make sure that the script is as small as possible. You can use any legal combination of signatures to redeem the transaction but make sure that all combinations would have worked.

In [ ]:
# Create three customer's keys
# Your code

In [ ]:
# Input script - setup the contract to require the signature of your partner and at least one of the three addresses you just created.
# Your code

In [ ]:
# Redemption Script - your partner and one of the customers must sign.
# Your code